**Project MET5** 
Lallu 
Jyoti
Ina
Magda

Burnt Areas in Burkina Faso

Import of libraries:

In [1]:
import datacube
from datetime import datetime
from datetime import timedelta
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from zipfile import ZipFile
import geopandas as gpd
from pyproj import Proj, transform
from datacube.utils.cog import write_cog
from odc.ui import with_ui_cbk

from deafrica_tools.datahandling import load_ard, mostcommon_crs
from deafrica_tools.plotting import rgb, display_map
from deafrica_tools.bandindices import calculate_indices

Create connection to WASCAL data cube with a unique app name.

In [2]:
dc = datacube.Datacube(app="Burnt_area_mapping")

View List of Products in the Datacube

In [3]:
dc.list_products()

,name,description,license,default_crs,default_resolution
name,,,,,
ls7_sr,ls7_sr,USGS Landsat 7 Collection 2 Level-2 Surface Re...,CC-BY-4.0,None,None
ls7_st,ls7_st,USGS Landsat 7 Collection 2 Level-2 Surface Te...,CC-BY-4.0,None,None
ls8_sr,ls8_sr,USGS Landsat 8 Collection 2 Level-2 Surface Re...,CC-BY-4.0,None,None
ls8_st,ls8_st,USGS Landsat 8 Collection 2 Level-2 Surface Te...,CC-BY-4.0,None,None
s2_l2a,s2_l2a,"Sentinel-2a and Sentinel-2b imagery, processed...",None,None,None
wofs_ls,wofs_ls,Historic Flood Mapping Water Observations from...,CC-BY-4.0,None,None


Look up specifications of product

In [6]:
product = "ls8_st"
measurements = dc.list_measurements()
measurements.loc[product]

,name,dtype,units,nodata,aliases,flags_definition
measurement,,,,,,
ST_B10,ST_B10,uint16,Kelvin,0,"[band_10, st, surface_temperature]",NaN
ST_TRAD,ST_TRAD,int16,W/(m2.sr.μm),-9999,"[trad, thermal_radiance]",NaN
ST_URAD,ST_URAD,int16,W/(m2.sr.μm),-9999,"[urad, upwell_radiance]",NaN
ST_DRAD,ST_DRAD,int16,W/(m2.sr.μm),-9999,"[drad, downwell_radiance]",NaN
ST_ATRAN,ST_ATRAN,int16,1,-9999,"[atran, atmospheric_transmittance]",NaN
ST_EMIS,ST_EMIS,int16,1,-9999,"[emis, emissivity]",NaN
ST_EMSD,ST_EMSD,int16,1,-9999,"[emsd, emissivity_stddev]",NaN
ST_CDIST,ST_CDIST,int16,Kilometers,-9999,"[cdist, cloud_distance]",NaN
QA_PIXEL,QA_PIXEL,uint16,bit_index,1,"[pq, pixel_quality]","{'snow': {'bits': 5, 'values': {'0': 'not_high..."


Data availibility

landsat 8_sr data available every 16 days\
for January/February 2020: 01-07, 01-23, 02-08, 02-24

In [ ]:
from odc.ui import DcViewer

dcv = DcViewer(dc,time='2020-01-07', center=(12,1), zoom=6)
dcv

Display Research Area 




In [ ]:
from dea_tools.plotting import display_map
display_map([-4.00 ,-3.79],[11.75 ,11.36], crs = 'EPSG:4620')

In [8]:
# Define the area of interest
lat = 11.564
lon = -3.861
buffer = 0.2

# Compute the bounding box for the study area
lat_range = (lat-buffer, lat+buffer)
lon_range = (lon-buffer, lon+buffer)

Define Fire Period

In [9]:
# Fire event date
fire_date = '2019-12-01'

# Length of baseline period
baseline_length = '3 months'

In [ ]:
display_map(x=lon_range, y=lat_range)

In [39]:
# Define load parameters
resolution = (-30, 30)
measurements = ['blue', 'green', 'red',
                'nir', 'swir_1', 'swir_2', 'pixel_quality', 'radiometric_saturation']
#collection_category='T1'
min_gooddata = 0.6

In [41]:
# Choose the Sentinel-2 products to load
products = ["ls8_sr"]

# Create a reusable query
query = {
    "x": lon_range,
    "y": lat_range,
    "resolution": resolution,
    "measurements": measurements
}

# Since this analysis calculates pixel areas,
# set the output projection to equal area projection EPSG:6933
output_crs = "EPSG:6933"

# Load all data in basline period avalible from ARD data
baseline_ard = load_ard(dc=dc,
                        products=products,
                        time=('2019-12-31', '2020-02-29'),
                        min_gooddata=min_gooddata,
                        output_crs=output_crs,
                        group_by='solar_day',
                        #collection_category=collection_category,
                        **query)

#
"""baseline_dc = dc.load(product=products,
                    output_crs=output_crs,
                     time=('2019-12-31', '2020-02-29'),
                     resampling=None,
                     skip_broken_datasets=False,
                     dask_chunks=None,
                     like=None,
                     fuse_func=None,
                     align=None,
                     datasets=None,
                     dataset_predicate=None,
                     progress_cbk=None,
                     **query)
                     """
                 

Using pixel quality parameters for USGS Collection 2
Finding datasets
    ls8_sr


/opt/conda/lib/python3.9/site-packages/deafrica_tools/datahandling.py:467: FutureWarning: xarray.ufuncs is deprecated. Instead, use numpy ufuncs directly.
  pq_mask = xr.ufuncs.logical_or(pq_mask, pq_mask)
/opt/conda/lib/python3.9/site-packages/xarray/core/dataarray.py:3086: FutureWarning: xarray.ufuncs is deprecated. Instead, use numpy ufuncs directly.
  f(self.variable, other_variable)
/opt/conda/lib/python3.9/site-packages/xarray/core/variable.py:2429: FutureWarning: xarray.ufuncs is deprecated. Instead, use numpy ufuncs directly.
  f(self_data, other_data) if not reflexive else f(other_data, self_data)


Counting good quality pixels for each time step
Filtering to 7 out of 7 time steps with at least 60.0% good quality pixels
Applying pixel quality/cloud mask
Re-scaling Landsat C2 data
Loading 7 time steps


"baseline_dc = dc.load(product=products,\n                    output_crs=output_crs,\n                     time=('2019-12-31', '2020-02-29'),\n                     resampling=None,\n                     skip_broken_datasets=False,\n                     dask_chunks=None,\n                     like=None,\n                     fuse_func=None,\n                     align=None,\n                     datasets=None,\n                     dataset_predicate=None,\n                     progress_cbk=None,\n                     **query)\n                     "

In [42]:
baseline_ard

<xarray.Dataset>
Dimensions:                 (time: 7, y: 1669, x: 1288)
Coordinates:
  * time                    (time) datetime64[ns] 2020-01-07T10:32:50.713671 ...
  * y                       (y) float64 1.491e+06 1.491e+06 ... 1.44e+06
  * x                       (x) float64 -3.918e+05 -3.918e+05 ... -3.532e+05
    spatial_ref             int32 6933
Data variables:
    blue                    (time, y, x) float32 0.07725 0.08749 ... 0.05836
    green                   (time, y, x) float32 0.1311 0.1406 ... 0.08944
    red                     (time, y, x) float32 0.1678 0.1865 ... 0.1258 0.1192
    nir                     (time, y, x) float32 0.2971 0.307 ... 0.1953 0.1837
    swir_1                  (time, y, x) float32 0.3699 0.3823 ... 0.299 0.2977
    swir_2                  (time, y, x) float32 0.2668 0.2915 ... 0.2343 0.2339
    pixel_quality           (time, y, x) uint16 21824 21824 ... 21824 21824
    radiometric_saturation  (time, y, x) uint16 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref

In [37]:
baseline_dc

<xarray.Dataset>
Dimensions:                 (time: 7, y: 1669, x: 1288)
Coordinates:
  * time                    (time) datetime64[ns] 2020-01-07T10:32:50.713671 ...
  * y                       (y) float64 1.491e+06 1.491e+06 ... 1.44e+06
  * x                       (x) float64 -3.918e+05 -3.918e+05 ... -3.532e+05
    spatial_ref             int32 6933
Data variables:
    blue                    (time, y, x) uint16 10082 10454 10801 ... 9409 9395
    green                   (time, y, x) uint16 12040 12387 ... 10530 10525
    red                     (time, y, x) uint16 13376 14055 ... 11848 11608
    nir                     (time, y, x) uint16 18077 18436 ... 14376 13951
    swir_1                  (time, y, x) uint16 20723 21174 ... 18147 18100
    swir_2                  (time, y, x) uint16 16974 17872 ... 15791 15777
    pixel_quality           (time, y, x) uint16 21824 21824 ... 21824 21824
    radiometric_saturation  (time, y, x) uint16 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref